In [1]:
import numpy as np
import mnist

import nn as n

In [2]:
conv = n.Conv(input_shape=(28, 28), conv_size=3, num_filters=8)
pool = n.MaxPool(input_shape=conv.output_shape, pool_size=2)
smax = n.SoftMax(input_shape=pool.output_shape, nodes=10)

print(conv)
print(pool)
print(smax)

def forward(image, label):

    # forward
    output = conv.forward(image/255-0.5)
    output = pool.forward(output)
    output = smax.forward(output)
    
    loss = -np.log(output[label])
    acc = 1 if label == np.argmax(output) else 0    
    
    return output, loss, acc

def train(image, label, lr=.005):
    
    out, loss, acc = forward(image, label)
    
    grad = np.zeros(10)
    grad[label] = -1/out[label]

    grad, grad_alpha, grad_beta = smax.backprop(grad)
    grad = pool.backprop(grad)
    _, grad_w = conv.backprop(grad)
    
    # gradient descent
    smax.alpha -= lr*grad_alpha
    smax.beta -= lr*grad_beta
    conv.w -= lr*grad_w
    
    return loss, acc

Conv: (28, 28) --> (26, 26, 8)
MaxPool: (26, 26, 8) --> (13, 13, 8)
SoftMax: (13, 13, 8) --> 10


In [3]:
# Own network

N = 1000
train_images = mnist.train_images()[:N]
train_labels = mnist.train_labels()[:N]
test_images = mnist.test_images()[:N]
test_labels = mnist.test_labels()[:N]

for epoch in range(3):

    print("--- Epoch {} ---".format(1+epoch))

    permutation = np.random.permutation(len(train_images))
    train_images = train_images[permutation]
    train_labels = train_labels[permutation]    

    loss = 0
    num_correct = 0
    for i, (image, label) in enumerate(zip(train_images, train_labels)):

        if i > 0 and i % 100 == 99:
            print('Step {:d}: Average Loss {:.2f}, Accuracy: {:.1f}%'.format(i//100, loss / 100, num_correct))
            num_correct = 0
            loss = 0

        l, acc = train(image, label)
        num_correct += acc
        loss += l

# Test 
loss = 0
num_correct = 0
for im, label in zip(test_images, test_labels):
    _, l, acc = forward(im, label)
    loss += l
    num_correct += acc

num_tests = len(test_images)
print('Test Loss:', loss / num_tests)
print('Test Accuracy:', num_correct / num_tests)

--- Epoch 1 ---
Step 0: Average Loss 2.20, Accuracy: 22.0%
Step 1: Average Loss 1.94, Accuracy: 36.0%
Step 2: Average Loss 1.37, Accuracy: 62.0%
Step 3: Average Loss 1.12, Accuracy: 65.0%
Step 4: Average Loss 0.83, Accuracy: 76.0%
Step 5: Average Loss 0.95, Accuracy: 63.0%
Step 6: Average Loss 0.68, Accuracy: 79.0%
Step 7: Average Loss 0.78, Accuracy: 71.0%
Step 8: Average Loss 0.58, Accuracy: 84.0%
Step 9: Average Loss 0.52, Accuracy: 87.0%
--- Epoch 2 ---
Step 0: Average Loss 0.58, Accuracy: 81.0%
Step 1: Average Loss 0.42, Accuracy: 89.0%
Step 2: Average Loss 0.58, Accuracy: 84.0%
Step 3: Average Loss 0.53, Accuracy: 83.0%
Step 4: Average Loss 0.54, Accuracy: 85.0%
Step 5: Average Loss 0.58, Accuracy: 78.0%
Step 6: Average Loss 0.35, Accuracy: 91.0%
Step 7: Average Loss 0.48, Accuracy: 81.0%
Step 8: Average Loss 0.63, Accuracy: 79.0%
Step 9: Average Loss 0.48, Accuracy: 89.0%
--- Epoch 3 ---
Step 0: Average Loss 0.63, Accuracy: 74.0%
Step 1: Average Loss 0.37, Accuracy: 87.0%
Step 2

In [4]:
import torch
import torch.utils.data as data_utils

train = data_utils.TensorDataset(
            torch.tensor(train_images[:, np.newaxis, ...]/255, dtype=torch.float), 
            torch.tensor(train_labels, dtype=torch.int64)
        )
test = data_utils.TensorDataset(
            torch.tensor(test_images[:, np.newaxis, ...]/255, dtype=torch.float), 
            torch.tensor(test_labels, dtype=torch.int64)
        )

train_loader = data_utils.DataLoader(train, batch_size=128, shuffle=True)
test_loader = data_utils.DataLoader(test, batch_size=N)


class Flatten(torch.nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

# Architecture
# net = torch.nn.Sequential(
#     torch.nn.Conv2d(1, 8, 3, bias=False),
#     torch.nn.MaxPool2d(kernel_size=2),
#     Flatten(),
#     torch.nn.Linear(in_features=8*13*13, out_features=10)
# )

net = torch.nn.Sequential(
    torch.nn.Conv2d(1, 32, 3),
    torch.nn.ReLU(),    
    torch.nn.Conv2d(32, 64, 3),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(2, 2),
    Flatten(),
    torch.nn.Linear(64*12*12, 128),
    torch.nn.ReLU(),
    torch.nn.Linear(128, 10)
)

# Loss and optimizer selections
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adadelta(net.parameters())

In [5]:
N = 5000
train_images = mnist.train_images()[:N]
train_labels = mnist.train_labels()[:N]
test_images = mnist.test_images()[:N]
test_labels = mnist.test_labels()[:N]

for epoch in range(12):
    for batch_idx, (data, target) in enumerate(train_loader):
        
        optimizer.zero_grad()      
        output = net(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        if False:
            print('Epoch {} batch {}: accuracy {:.0f}% loss {:.6f}'.format(
                1+epoch, 
                1+batch_idx, 
                100*output.argmax(dim=1).eq(target).sum().item() / float(target.shape[0]), 
                loss.item()
            ))

for data, target in test_loader:
    output = net(data)
    acc = 100*output.argmax(dim=1).eq(target).sum().item() / float(target.shape[0])
print("Test accuracy: {:.2f}%".format(acc))

Test accuracy: 88.90%
